# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [65]:
# Dependencies and Setup
import hvplot.pandas as hv
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [66]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-10.54,84,100,7.79,RU,1711875548
1,1,mar del plata,-38.0023,-57.5575,14.93,63,0,1.79,AR,1711874984
2,2,fort wellington,6.4000,-57.6000,26.28,82,98,2.76,GY,1711875549
3,3,thinadhoo,0.5333,72.9333,29.12,72,12,2.93,MV,1711875549
4,4,necochea,-38.5473,-58.7368,13.81,58,100,2.29,AR,1711875482


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [70]:
# %%capture --no-display

# import geoviews as gv
# import geoviews.tile_sources as gvts
# # Configure the map plot
# points = gv.Points(city_data_df, ['Lng', 'Lat'], ['Humidity'])

# # Customize the plot by setting the tile source and point size
# map_plot = points.opts(
#     color='blue', size='Humidity',cmap='Blues', alpha=0.02,
#     tools=['hover'], width=800, height=600, title='City Humidity Map',
#     backend='bokeh'
# ) * gvts.OSM

# # Create a scatter plot using hvplot.scatter
# scatter_plot = city_data_df.hvplot.scatter(x='Lng',y='Lat', c='Humidity')

# # Display the map
# map_plot*scatter_plot


:Overlay
   .Points.I  :Points   [Lng,Lat]   (Humidity)
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Scatter.I :Scatter   [Lng]   (Lat,Humidity)

In [93]:
%%capture --no-display

import geoviews as gv
import geoviews.tile_sources as gvts
# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",   #OpenStreetMap (OSM) tiles
    frame_width = 700,
    frame_height = 500)

# # Customize the plot by setting the tile source and point size
# map_plot = points.opts(
#     color="city", size="Humidity", alpha=0.5,scale=1,
#     tools=['hover'], width=800, height=600, title='City Humidity Map',
#     backend='bokeh'
# ) * gvts.OSM

# # Create a scatter plot using hvplot.scatter
# scatter_plot = city_data_df.hvplot.scatter(x='Lng',y='Lat', c='Humidity')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values
if not city_data_df.empty:
    criteria = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
    filtered_cities = city_data_df[criteria]

# Drop any rows with null values
    filtered_cities = filtered_cities.dropna()

# Display sample data
    print(filtered_cities)
else:
    print("No data in the DataFrame. Please check your data source.")


     City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
160      160       cobram -35.9167  145.6500     25.24        22           0   
169      169       merida  20.9667  -89.6167     21.04        79           0   
173      173  lake albert -35.1667  147.3833     26.03        29           0   
254      254  tamanrasset  22.7850    5.5228     26.95        19           0   
260      260       callao -12.0667  -77.1500     24.29        83           0   
448      448         anlu  31.1661  112.5831     26.80        50           0   

     Wind Speed Country        Date  
160        2.59      AU  1711875622  
169        2.89      MX  1711875526  
173        2.13      AU  1711875628  
254        2.06      DZ  1711875666  
260        3.60      PE  1711875120  
448        3.54      CN  1711875754  


### Step 3: Create a new DataFrame called `hotel_df`.

In [82]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']])


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
160,cobram,AU,-35.9167,145.6500,22,
169,merida,MX,20.9667,-89.6167,79,
173,lake albert,AU,-35.1667,147.3833,29,
254,tamanrasset,DZ,22.7850,5.5228,19,
260,callao,PE,-12.0667,-77.1500,83,
448,anlu,CN,31.1661,112.5831,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [83]:
# Set parameters to search for a hotel
radius = 10000

# Set the parameters for the type of place

params = { "categories":"accommodation.hotel",
    "conditions":"access.not_specified",
    "limit":20,
    "apiKey":geoapify_key   
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat=hotel_df.loc[index,'Lat']
    Lng=hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cobram - nearest hotel: No hotel found
merida - nearest hotel: Hotel Mejorada Mérida
lake albert - nearest hotel: The Astor
tamanrasset - nearest hotel: فندق الأمان
callao - nearest hotel: Casa Ronald
anlu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
160,cobram,AU,-35.9167,145.6500,22,No hotel found
169,merida,MX,20.9667,-89.6167,79,Hotel Mejorada Mérida
173,lake albert,AU,-35.1667,147.3833,29,The Astor
254,tamanrasset,DZ,22.7850,5.5228,19,فندق الأمان
260,callao,PE,-12.0667,-77.1500,83,Casa Ronald
448,anlu,CN,31.1661,112.5831,50,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [95]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng", "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",   #OpenStreetMap (OSM) tiles
    frame_width = 700,
    frame_height = 500,                                
    hover_cols=["City", "Country", "Hotel Name"])
# map_plot = hotel_df.hvplot.points(
#     "Lat",
#     "Lng",
#     geo=True, #It could be used to enable geospatial features or to ensure that the map is rendered correctly for geographic data.
#     tiles="EsriImagery",  #"EsriImagery" likely refers to map tiles provided by Esri (Environmental Systems Research Institute) that display satellite imagery or aerial photographs as the base layer of the map.
#     frame_width=800,
#     frame_height=600,
#     hover_cols=["City", "Country", "Hotel Name"]
# )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)